In [10]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPool2D,Flatten,Dense,Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# train.csv 파일 읽어오기 (label)
f= open('train.csv','r')
label_data=pd.read_csv(f,header=0)
seq=label_data[['label']].to_numpy()

# 이미지 읽어오기 (data)
images = np.zeros((5000,64,64,3))
for i in range(5000):
    images[i]=image.load_img('train/train_img_{0}.jpg'.format(i))

# 훈련집합과 테스트집합으로 분할
x_train,x_test,y_train,y_test=train_test_split(images,seq,stratify=seq,test_size=0.2)

x_train=x_train.astype(np.float32)/255.0 # 정규화
x_test=x_test.astype(np.float32)/255.0
y_train=tf.keras.utils.to_categorical(y_train,5) # 원핫코드로 변환(레이블은 0,1,2,3,4 총 5개 부류)
y_test=tf.keras.utils.to_categorical(y_test,5)

# 드랍아웃이 없는게 성능이 더 좋음
cnn=Sequential()
cnn.add(Conv2D(64,(3,3),activation='relu',input_shape=(64,64,3)))
cnn.add(Conv2D(64,(3,3),activation='relu'))
cnn.add(MaxPool2D(pool_size=(2,2)))
cnn.add(Conv2D(64,(3,3),activation='relu'))
cnn.add(Conv2D(64,(3,3),activation='relu'))
cnn.add(MaxPool2D(pool_size=(2,2)))
cnn.add(Flatten())
cnn.add(Dense(512,activation='relu'))
cnn.add(Dense(5,activation='softmax'))

cnn.compile(loss='categorical_crossentropy',optimizer=Adam(),metrics=['accuracy'])
# 데이터 증대
generator=ImageDataGenerator(width_shift_range=0.1,height_shift_range=0.1,horizontal_flip=True)
cnn.fit_generator(generator.flow(x_train,y_train,batch_size=128),epochs=30,validation_data=(x_test,y_test),verbose=2)

res=cnn.evaluate(x_test,y_test,verbose=0)
print("정확률은",res[1]*100)



Epoch 1/30


/var/folders/yy/5xxknbnj0sx3x_39kz04q09w0000gn/T/ipykernel_56018/3920414413.py:44: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  cnn.fit_generator(generator.flow(x_train,y_train,batch_size=128),epochs=30,validation_data=(x_test,y_test),verbose=2)
2022-05-24 22:15:52.304456: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-05-24 22:15:57.662241: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


32/32 - 6s - loss: 1.4809 - accuracy: 0.3430 - val_loss: 1.3603 - val_accuracy: 0.4560 - 6s/epoch - 192ms/step
Epoch 2/30
32/32 - 5s - loss: 1.2209 - accuracy: 0.5250 - val_loss: 1.0422 - val_accuracy: 0.6360 - 5s/epoch - 172ms/step
Epoch 3/30
32/32 - 5s - loss: 1.0389 - accuracy: 0.6095 - val_loss: 0.9936 - val_accuracy: 0.6280 - 5s/epoch - 171ms/step
Epoch 4/30
32/32 - 6s - loss: 0.9092 - accuracy: 0.6595 - val_loss: 0.7048 - val_accuracy: 0.7350 - 6s/epoch - 177ms/step
Epoch 5/30
32/32 - 6s - loss: 0.7808 - accuracy: 0.7100 - val_loss: 0.6679 - val_accuracy: 0.7510 - 6s/epoch - 172ms/step
Epoch 6/30
32/32 - 5s - loss: 0.7491 - accuracy: 0.7205 - val_loss: 0.6439 - val_accuracy: 0.7520 - 5s/epoch - 170ms/step
Epoch 7/30
32/32 - 5s - loss: 0.6611 - accuracy: 0.7490 - val_loss: 0.6908 - val_accuracy: 0.7410 - 5s/epoch - 169ms/step
Epoch 8/30
32/32 - 5s - loss: 0.6228 - accuracy: 0.7595 - val_loss: 0.5393 - val_accuracy: 0.7960 - 5s/epoch - 170ms/step
Epoch 9/30
32/32 - 5s - loss: 0.564

In [11]:
cnn.save('final_cnn.h5')

In [28]:
cnn=tf.keras.models.load_model("final_cnn.h5")
cnn.summary()

f= open('train.csv','r')
label_data=pd.read_csv(f,header=0)
seq=label_data[['label']].to_numpy()

# 이미지 읽어오기 (data)
images = np.zeros((5000,64,64,3))
for i in range(5000):
    images[i]=image.load_img('train/train_img_{0}.jpg'.format(i))

# 훈련집합과 테스트집합으로 분할
x_train,x_test,y_train,y_test=train_test_split(images,seq,stratify=seq,test_size=0.2)

x_train=x_train.astype(np.float32)/255.0 # 정규화
x_test=x_test.astype(np.float32)/255.0
y_train=tf.keras.utils.to_categorical(y_train,5) # 원핫코드로 변환(레이블은 0,1,2,3,4 총 5개 부류)
y_test=tf.keras.utils.to_categorical(y_test,5)

res=cnn.evaluate(x_test,y_test,verbose=0)
print("정확률은",res[1]*100)

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_32 (Conv2D)          (None, 62, 62, 64)        1792      
                                                                 
 conv2d_33 (Conv2D)          (None, 60, 60, 64)        36928     
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 30, 30, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_34 (Conv2D)          (None, 28, 28, 64)        36928     
                                                                 
 conv2d_35 (Conv2D)          (None, 26, 26, 64)        36928     
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 13, 13, 64)       0         
 g2D)                                                 

2022-05-24 22:24:00.117180: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


정확률은 92.80000329017639
